Modify the regression scratch code in our lecture such that:

- Implement early stopping in which if the absolute difference between old loss and new loss does not exceed certain threshold, we abort the learning.

- Implement options for stochastic gradient descent in which we use only one sample for training.  Make sure that sample does not repeat unless all samples are read at least once already.

- Add options for mini-batch gradient descent.

- Put everything into class.

In [23]:
from sklearn.datasets import load_boston

boston = load_boston()
X = boston.data
m = X.shape[0]  
n = X.shape[1]
y = boston.target
assert m == y.shape[0]       

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


import numpy as np
intercept = np.ones((X_train.shape[0], 1))
X_train = np.concatenate((intercept, X_train), axis=1)        #X_train = intercept naja 
intercept = np.ones((X_test.shape[0], 1))
X_test = np.concatenate((intercept, X_test), axis=1)


# class
class LinearRegression:
#1 Batch GD
    def __init__(self, alpha = 0.0001, max_iter = 10000, old_loss = 10000, tol = 0.0001, method="batch"):
        self.alpha = alpha
        self.max_iter = max_iter
        self.old_loss = old_loss
        self.tol = tol               
        self.method = method
        
    def fit(self, X, y, X_test, y_test):
        self.theta = np.zeros(X.shape[1])
        iter_stop = 0
        list_solo = []            #without replacement naja, no same sam  
        
        for i in range(self.max_iter):
            if self.method == "sto":
                i = np.random.randint(X.shape[0])
                while i in list_solo :
                    i = np.random.randint(X.shape[0])
                X_train = X[i, :].reshape(1, -1)
                y_train = y[i]
                list_solo.append(i)
                if len(list_solo) == X.shape[0]:
                    list_solo = []
            elif self.method == "mini-batch":
                random_index = np.random.randint(X.shape[0])

                mini_batch_X = X[random_index:random_index+100]
                mini_batch_y = y[random_index:random_index+100]
                X_train = mini_batch_X
                y_train = mini_batch_y
            else:
                X_train = X
                y_train = y
            
            #1. prediction = X @ w
            yhat = self.h_theta(X_train)
            
            #2. error
            error = yhat - y_train
        
            # stop
            new_loss = self.mse(yhat, y_train)
            if self.final_loss(new_loss, self.old_loss, self.tol): 
                iter_stop = i
                break
            self.old_loss = new_loss
            
             #3. grad = X.T @ error
            grad = self.gradient(X_train, error)
            
            # 4.update: w  =  w     -   alpha    * grad
            self.theta = self.theta - self.alpha * grad
            
            if i%10 == 0:
                yhat = model.h_theta(X_test)                   # TA มองผ่านๆนะคะ : ถ้าเอา if % ออก ก็จิได้ mse สุดท้าย
                mse_batch = model.mse(yhat, y_test)
                print(mse_batch)  
            
    def h_theta(self, X):
        return X @ self.theta

    def mse(self, yhat, y):
        return ((yhat - y)**2 / yhat.shape[0]).sum()     #sum mse (ถ้าไม่ sum, print นางจะอลัง)

    def gradient(self, X, error):
        return X.T @ error
    
    def final_loss(self, old_loss, new_loss, tol):
        return np.abs(new_loss - old_loss) < tol        # continued until the scores in the last  
                                                        # do not improve by atleast tol

    
#2 Sto --> random single sam     
model = LinearRegression(method="sto") 
model.fit(X_train, y_train, X_test, y_test)

# accu
yhat = model.h_theta(X_test)
mse_sto = model.mse(yhat, y_test)
print(mse_sto)


#3 minibatch --> subset of observa to up the GD
model = LinearRegression(method="mini-batch") 
model.fit(X_train, y_train, X_test, y_test)

# accu
yhat = model.h_theta(X_test)
mse_mini = model.mse(yhat, y_test)
print(mse_mini)

568.7074282488504
568.6224349553576
567.8385545575761
565.7557468417089
565.5295770482369
563.6967812981823
563.0412509469716
561.9030412615647
560.7253635073732
559.5428044679054
558.8352352695574
558.4249824083415
555.3909241186877
547.8560646203282
547.5126057625648
546.6152742965421
546.0471985932958
545.2457050592751
542.2737189160868
540.5400979745021
540.5258812312081
539.4025420388075
538.6905782989688
537.8899976239377
536.487644173448
536.432978300279
535.9887054244209
533.0652917489264
531.769178999572
531.1693108375061
530.0691490075246
528.3642624939303
528.3266113833919
526.8853347742767
526.1986941908034
525.9109430624735
522.8151529834079
522.6038768468427
521.4072195984676
520.9161001801849
520.6387184977903
519.744580223725
518.6222082224913
518.0294654594459
517.5885326600348
516.05912409846
512.1751651813181
511.9616354330534
510.0644151261809
509.9898006668851
509.1185157609506
508.4565300086263
507.0934793362028
506.65527502131005
505.0525904513301
504.24194655598